# L5: Content Creation at Scale

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
# Warning control
import warnings
import yaml
from crewai import Agent, Task, Crew
# Load environment variables
import os
from dotenv import load_dotenv
load_dotenv(r"F:\Social AI\multiagent-framework\.venv\.env")
warnings.filterwarnings('ignore')

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Creating Structured Output

In [2]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [8]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [4]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

In [14]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
groq_llm = "groq/llama3-8b-8192"

## Creating Crew, Agents, and Tasks

In [15]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

2024-11-29 22:33:51,996 - 16040 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Kicking off the Crew

In [16]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Don't use a lot of resources just two. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.



# Agent: Lead Market Analyst
## Thought: Thought: I need to gather the latest news and updates on Inflation in the US and its impact on the stock market in 2024.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"latest news on US inflation and its impact on the stock market 2024\"}"
## Tool Output: 

Search results: Title: The stock m

## Social Content

In [17]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
🚨 Inflation in the US is projected to decline to
2.7% by mid-2024! With market volatility
anticipated, it's crucial to adjust your
investment strategies. #Finances #Investing
#Inflation
--------------------------------------------------
LinkedIn
Inflation is set to moderate, but what does this
mean for the stock market? As we approach 2024,
investors have reason to be hopeful with CPI rates
expected around 2.7%. Learn how to navigate these
changes effectively and position your portfolio
for success. [Link to Blog Post]
--------------------------------------------------
Instagram
📉📈 Inflation down to 2.7% in 2024? That's good
news for stocks! Stay ahead of the curve and
invest wisely. Check our latest blog for insights
on market trends. #InvestSmart #MarketTrends
#Finance2024
--------------------------------------------------
Visual Infographic
A detailed infographic could show the trajectory
of inflation alongside stock market performance
metrics. This infographic can highlight

## Blog Post

In [18]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

# Navigating the Market in a Time of Inflation: Insights for 2024

## Introduction
As we head into 2024, the landscape of the US economy is painted with the hues of inflation and market volatility. With the current inflation rate projected to recede significantly, it is vital for investors to understand how these economic changes affect stock market performance. This article will delve into inflation trends, the Federal Reserve's influence, market reactions, and actionable insights for investors navigating this complex environment.

## Understanding Inflation Trends
The recent statistics show that inflation rates have shown a downward trajectory, decreasing from peaks that alarmed markets back in 2023. In July, the Consumer Price Index reported an inflation rate of only 2.7%, down from an unsettling 7.1% the previous year. This notable decline has stirred optimism among many market analysts, suggesting a period of stability ahead.

The implications of this decline are noteworthy. For instance, a lower inflation rate typically translates to increased consumer spending power, which can bolster economic growth. Investors should monitor these trends closely, as they may indicate opportunities for stock market investments in sectors likely to benefit from a recovering economy.

## The Fed's Role in the Market Dynamics
The Federal Reserve's monetary policies play a crucial role in shaping market conditions. Their decision to taper asset purchases and reduce interest rates is seen as a pivotal moment that may boost investor confidence and push stock prices higher. As of now, the yield on 10-year US Treasury bonds, which impacts investment flow, is stabilizing, making equities more appealing than fixed income securities.

It’s essential to note that while lower interest rates may stimulate consumer and business lending, they can also lead to increased risk-taking in the stock market. Investors should be cautious and evaluate how these changes impact their portfolio strategies.

## Market Reactions and Insights
Interestingly, the stock market has exhibited resilience throughout these inflationary pressures. The S&P 500 has consistently achieved record highs, indicating that investors are adapting to the new normal and re-evaluating their strategies. Many are now leaning towards dividend-paying stocks, viewing them as safer bets amid fluctuating interest rates.

Moreover, sectors such as technology and healthcare have shown robust performance despite inflation concerns. As the market reacts to these economic shifts, investors are advised to remain agile, ready to pivot as new data emerges and conditions change.

## Data Visualizations
To better illustrate these trends, we have included several graphs:

- **Figure 1: Inflation Rate vs. Stock Market Performance** - Showing the correlation between inflation decline and stock market recoveries. 
- **Figure 2: Federal Reserve Interest Rates vs. Stock Market Performance** - An analysis of the relationship between rate adjustments and stock performance indicators.
- **Figure 3: Consumer Price Index Over Time** - Highlighting historical data to understand the inflationary trajectory.

These visual aids aim to provide a clearer picture of how inflation trends correlate with stock movement, enabling investors to make more informed decisions.

## Investor Recommendations
As we move forward, here are key takeaways for investors:
1. **Stay Informed**: Keep an eye on reports from trusted economic analysts to gauge the changing landscape. Subscribe to financial news outlets and consider following economic indicators that reliably predict shifts.
2. **Diversify Investments**: Consider diversifying into sectors that traditionally perform well in lower inflation environments, such as technology and consumer goods. Diversification can help mitigate risk and capitalize on varied market performances.
3. **Focus on Dividends**: With interest rates projected to remain low, dividend-paying stocks could provide steady returns and mitigate risks. Investors should prioritize companies with a history of consistent dividend growth.

## Conclusion
The intersection of inflation and the stock market presents a complex but navigable path for investors in 2024. Understanding these dynamics will empower you to make informed investment decisions and adjust your strategies to align with the shifting economic conditions. Success in this environment requires vigilance, adaptability, and a well-informed approach to portfolio management.